imports
-

In [1]:
import sys
sys.version

'3.9.13 (main, Aug 25 2022, 23:26:10) \n[GCC 11.2.0]'

In [2]:
#math
import numpy as np
#capture
import cv2
import wave
#utils
from tqdm import tqdm
from IPython.display import Image

In [3]:
#neural
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
print(tf.__version__)

2022-10-14 02:15:17.485382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 02:15:17.671807: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-14 02:15:18.299186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alex/miniconda3/envs/tf/lib/python3.9/site-packages/cv2/../../lib64::/home/alex/miniconda3/envs/tf/lib/
2022-10-14 02:15:18.299280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not 

2.10.0


system info
-

import data
-

In [4]:
def reshape(frame):
    global WIDTH, HEIGHT
    newframe = np.zeros(WIDTH*HEIGHT).reshape(HEIGHT, WIDTH)
    if frame.shape[0] == HEIGHT and frame.shape[1] == WIDTH:
        return frame
    elif frame.shape[0] <= HEIGHT and frame.shape[1] <= WIDTH:
        newframe[:frame.shape[0],:frame.shape[1]] = frame
    elif frame.shape[0] <= HEIGHT:
        print('HEIGHT')
        newframe[:frame.shape[0],:] = frame[:, :WIDTH]
    elif frame.shape[1] <= WIDTH:
        print('WIDTH')
        newframe[:,:frame.shape[1]] = frame[:HEIGHT, :]
    else:
        newframe = frame[:HEIGHT, :WIDTH]
    return newframe

In [5]:
WIDTH = int(720)
HEIGHT = int(625)

def normilizeFrame(frame):
    global WIDTH, HEIGHT
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #change dimentions(choose 1 of 2 options)
    #frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    frame = reshape(frame)
    
    return frame
    #return frame / 255.0

fileName = 'testfiles/pal/testaud'

cap = cv2.VideoCapture(fileName + '.avi')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret, frame = cap.read()
if not ret:
    print('error reading frame')

def getNext():
    global WIDTH, HEIGHT
    if cap.grab():
        ret, frame = cap.retrieve()
        if not ret:
            print('error reading frame')
            return False, np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT)
        return True, normilizeFrame(frame)
    else:
        return False, np.zeros(WIDTH*HEIGHT).reshape(WIDTH, HEIGHT)

gray = normilizeFrame(frame)

In [6]:
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frameCount)

#tech calc

#audio in pcm have 44100 samples in second per channel, respectively we have 88200 samples
#per frame we have 44100/25(pal) = 1764 samples per frame(for single channel)

samplesPerFrame = int(44100/cap.get(cv2.CAP_PROP_FPS))
print(samplesPerFrame)

2381
1764


create dataset

In [7]:
dir = "./dataset/"

In [8]:
frames = len(list(tf.data.Dataset.list_files(f'{dir}*.png').as_numpy_iterator())) -1
mainAudioFile = wave.open(f'{dir}audio.wav', mode='r')

def gen():
    global frames, mainAudioFile, samplesPerFrame
    currentFrame = 1
    
    #audio
    mainAudioFile.rewind()
    
    if not (mainAudioFile.getparams()[0] == 2 and mainAudioFile.getparams()[2] == 44100):
        print('wrong audiofile params')
        sys.exit(1)
    
    #video
    
    while currentFrame < frames:
        #audio
        frame = mainAudioFile.readframes(samplesPerFrame)
        
        if len(frame) < samplesPerFrame*2:
            frame = frame + b'\x00' * ((samplesPerFrame*4)-len(frame))
        
        #video
        topframe=normilizeFrame(cv2.imread(f'{dir}{currentFrame}.png'))
        bottomframe=normilizeFrame(cv2.imread(f'{dir}{currentFrame+1}.png'))
        vframe = cv2.vconcat([topframe, bottomframe])
        
        yield vframe[np.newaxis is None,:,:], tf.io.decode_raw(frame, tf.int16, little_endian=True)
        currentFrame = currentFrame+1
    

2022-10-14 02:15:19.128619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 02:15:19.754954: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 910 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [20]:
tf.TensorArray(vframe)

ValueError: Size must be declared for TensorArrays when eager execution is enabled.

In [9]:
dataset=tf.data.Dataset.from_generator(
    gen, 
    output_signature=(tf.TensorSpec(shape=(None, (HEIGHT*2), WIDTH), dtype=tf.uint8, name='image'),
                      tf.TensorSpec(shape=(3528,), dtype=tf.int16, name='audio'),
                      #tf.TensorSpec(shape=(1), dtype=tf.int32, name='frameNum')
                     )
).cache('./dataset/cache/')#.repeat()


#testcode
-

#create model
-

In [10]:
input = keras.Input(shape=((HEIGHT*2), WIDTH), name = 'image')

In [11]:
x = tf.keras.layers.Rescaling(1./255, name='normalize')(input)

In [12]:
lr = 1
x = tf.keras.layers.Conv1D(128, 3, activation='relu', name=f'conv1d_{lr}', input_shape=(WIDTH, (HEIGHT*2)))(x)
x = tf.keras.layers.MaxPooling1D(name=f'MaxPooling1D_{lr}')(x)
lr = lr + 1
x = tf.keras.layers.Conv1D(128, 3, activation='relu', name=f'conv1d_{lr}', input_shape=(WIDTH, (HEIGHT*2)))(x)
x = tf.keras.layers.MaxPooling1D(name=f'MaxPooling1D_{lr}')(x)
lr = lr + 1
x = tf.keras.layers.Conv1D(128, 3, activation='relu', name=f'conv1d_{lr}', input_shape=(WIDTH, (HEIGHT*2)))(x)
x = tf.keras.layers.MaxPooling1D(name=f'MaxPooling1D_{lr}')(x)
lr = lr + 1
x = tf.keras.layers.Conv1D(128, 3, activation='relu', name=f'conv1d_{lr}', input_shape=(WIDTH, (HEIGHT*2)))(x)
x = tf.keras.layers.MaxPooling1D(name=f'MaxPooling1D_{lr}')(x)

In [13]:
x = tf.keras.layers.Flatten(name='Flatten')(x)

In [14]:
x = tf.keras.layers.Dense((samplesPerFrame*2), activation='relu', bias_initializer='zeros', use_bias=True, name='layer_1')(x)

In [15]:
output = tf.keras.layers.Dense((samplesPerFrame*2), activation='relu', bias_initializer='zeros', use_bias=True, name='audio')(x)

In [16]:
model = keras.Model(input, output)
model.summary()

tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 1250, 720)]       0         
                                                                 
 normalize (Rescaling)       (None, 1250, 720)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1248, 128)         276608    
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 624, 128)         0         
 D)                                                              
                                                                 
 conv1d_2 (Conv1D)           (None, 622, 128)          49280     
                                                                 
 MaxPooling1D_2 (MaxPooling1  (None, 311, 128)         0         
 D)                                                          

compile and learn
-

In [17]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [18]:
loss = 1
while loss >= 0.01:
    fitHistory = model.fit(dataset, epochs=1, batch_size=1)
    loss = fitHistory.history['loss'][0]

2022-10-14 02:15:20.636088: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-10-14 02:15:22.735433: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-10-14 02:15:23.179985: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-10-14 02:15:23.180813: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at conv_ops.cc:1134 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1: UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/stream_executor/cuda/cuda_dnn.cc(4023): 'cudnnConvolutionForward( cudnn.handle(), alpha, 

NotFoundError: Graph execution error:

Detected at node 'model/conv1d_1/Conv1D' defined at (most recent call last):
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/alex/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/alex/miniconda3/

save
-

In [ ]:
modelP2.save("model")